ここではvalidationデータでテストをする

In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn import decomposition
from sklearn import preprocessing
from sklearn import manifold
from sklearn import metrics
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import urllib.request
import shutil
import zipfile
from scipy.stats import mode

訓練データ

In [14]:
fold = "../train_raw_npy/"
# .npyファイルからデータをロード
train_x = np.load(f'{fold}x.npy')
train_y = np.load(f"{fold}label.npy")
from scipy.stats import mode
train_y = mode(train_y, axis=1).mode.flatten()

# データの形状を表示
print("Shape of the loaded x data:", train_x.shape)
print("Shape of the loaded label data:", train_y.shape)

Shape of the loaded x data: (196072, 500)
Shape of the loaded label data: (196072,)


テストデータ

In [5]:
fold = "../validation_raw_npy/"
# .npyファイルからデータをロード
test_x = np.load(f'{fold}x.npy')
test_y = np.load(f"{fold}label.npy")
from scipy.stats import mode
test_y = mode(test_y, axis=1).mode.flatten()

# データの形状を表示
print("Shape of the loaded x data:", test_x.shape)
print("Shape of the loaded label data:", test_y.shape)

Shape of the loaded x data: (28789, 500)
Shape of the loaded label data: (28789,)


In [15]:
print(pd.Series(train_y).value_counts())
print(pd.Series(test_y).value_counts())

5.0    31733
7.0    31253
6.0    28326
2.0    24544
1.0    24456
8.0    23834
4.0    23473
3.0     8453
Name: count, dtype: int64
1.0    5967
2.0    5225
7.0    4362
8.0    4342
5.0    4095
4.0    2407
6.0    1836
3.0     555
Name: count, dtype: int64


In [16]:
# 分割後のデータの形状を表示
print("Shape of train_x:", train_x.shape)
print("Shape of test_x:", test_x.shape)
print("Shape of train_y:", train_y.shape)
print("Shape of test_y:", test_y.shape)

Shape of train_x: (196072, 500)
Shape of test_x: (28789, 500)
Shape of train_y: (196072,)
Shape of test_y: (28789,)


In [17]:
print(np.unique(train_y))
print(np.unique(test_y))

[1. 2. 3. 4. 5. 6. 7. 8.]
[1. 2. 3. 4. 5. 6. 7. 8.]


In [9]:


clf_train_validation = BalancedRandomForestClassifier(
    n_estimators=2000,
    replacement=True,
    sampling_strategy='not minority',
    oob_score=True,
    n_jobs=8,
    random_state=42,
    verbose=1
)
clf_train_validation.fit(train_x, train_y)
#clf.fit(sampled_x_random,sampled_label_random) # 訓練データを使ってまずランダムフォレストを学習している。

c:\Users\kaiha\anaconda3\envs\labo_Okita\lib\site-packages\imblearn\ensemble\_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    3.3s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   17.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:   40.4s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:  2.9min
[Parallel(n_jobs=8)]: Done 2000 out of 2000 | elapsed:  3.3min finished


BalancedRandomForestClassifier(n_estimators=2000, n_jobs=8, oob_score=True,
                               random_state=42, replacement=True,
                               sampling_strategy='not minority', verbose=1)

In [11]:
Y_test_pred = clf_train_validation.predict(test_x) # 学習器の性能をみる テストデータで
print('\nClassifier performance')
print('Out of sample:\n', metrics.classification_report(test_y, Y_test_pred, zero_division=0))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    2.3s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    3.4s



Classifier performance
Out of sample:
               precision    recall  f1-score   support

         1.0       0.27      0.64      0.38      5967
         2.0       0.70      0.39      0.50      5225
         3.0       0.40      0.63      0.49       555
         4.0       0.12      0.03      0.04      2407
         5.0       0.23      0.23      0.23      4095
         6.0       0.04      0.04      0.04      1836
         7.0       0.17      0.09      0.12      4362
         8.0       0.24      0.08      0.12      4342

    accuracy                           0.28     28789
   macro avg       0.27      0.27      0.24     28789
weighted avg       0.30      0.28      0.25     28789



[Parallel(n_jobs=8)]: Done 2000 out of 2000 | elapsed:    3.9s finished


# 学習２

In [25]:
indices = np.random.choice(train_x.shape[0], len(train_x), replace=False)
sampled_x_random = train_x[indices]
sampled_label_random = train_y[indices]

In [27]:


clf_train_validation_all = BalancedRandomForestClassifier(
    n_estimators=500,
    replacement=True,
    sampling_strategy='not minority',
    oob_score=True,
    n_jobs=5,
    random_state=42,
    verbose=1
)
#clf_train_validation_all.fit(train_x, train_y)
clf_train_validation_all.fit(sampled_x_random,sampled_label_random) # 訓練データを使ってまずランダムフォレストを学習している。

c:\Users\kaiha\anaconda3\envs\labo_Okita\lib\site-packages\imblearn\ensemble\_forest.py:601: FutureWarning: The default of `bootstrap` will change from `True` to `False` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `False` to silence this warning and adopt the future behaviour.
  warn(
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  1.3min
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:  6.0min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed: 13.8min
[Parallel(n_jobs=5)]: Done 500 out of 500 | elapsed: 15.7min finished


BalancedRandomForestClassifier(n_estimators=500, n_jobs=5, oob_score=True,
                               random_state=42, replacement=True,
                               sampling_strategy='not minority', verbose=1)

In [28]:
Y_test_pred = clf_train_validation_all.predict(test_x) # 学習器の性能をみる テストデータで
print('\nClassifier performance')
print('Out of sample:\n', metrics.classification_report(test_y, Y_test_pred, zero_division=0))

[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:    0.5s
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed:    1.3s
[Parallel(n_jobs=5)]: Done 500 out of 500 | elapsed:    1.5s finished



Classifier performance
Out of sample:
               precision    recall  f1-score   support

         1.0       0.30      0.66      0.41      5967
         2.0       0.75      0.49      0.60      5225
         3.0       0.85      0.62      0.72       555
         4.0       0.21      0.03      0.06      2407
         5.0       0.24      0.24      0.24      4095
         6.0       0.04      0.04      0.04      1836
         7.0       0.31      0.21      0.25      4362
         8.0       0.28      0.13      0.18      4342

    accuracy                           0.33     28789
   macro avg       0.37      0.30      0.31     28789
weighted avg       0.36      0.33      0.31     28789

